# **Install package**

In [0]:
!pip install mlxtend==0.17.0
from mlxtend.plotting import plot_confusion_matrix

# **Import library**

In [0]:
import pandas as pd
import cv2
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from keras.layers import Activation, Dropout, Conv2D, Dense, MaxPooling2D, GlobalMaxPooling2D, Flatten, Input, BatchNormalization
from keras.models import Sequential, Model, load_model
from keras.regularizers import l2
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from google.colab.patches import cv2_imshow
from sklearn.metrics import *
import itertools
from keras.applications import InceptionV3, ResNet50
from google.colab import drive

# **Mount google drive**

In [0]:
drive.mount('/content/drive')

# **Set Path**

In [0]:
BASE_DIR = '/content/drive/My Drive/Google Colab/Emotional_Recognition/'
fer2013_path = BASE_DIR + 'fer2013/fer2013.csv'
ferplus_path = BASE_DIR + 'ferplus/'
emotion_model_path = BASE_DIR + 'models/cnn.hdf5'
detection_model_path = BASE_DIR + 'haarcascade_files/haarcascade_frontalface_default.xml'

# **Load dataset and preprocess**

In [0]:
def load_fer2013(shape = (48,48)):
    data = pd.read_csv(fer2013_path)
    pixels = data['pixels'].tolist()
    faces = []
    for pixel_sequence in pixels:
        face = list(map(int, pixel_sequence.split()))
        face = np.asarray(face).reshape(48, 48)
        img = face.astype('float32')
        if shape != (48,48):
          img = cv2.resize(img, shape)
        faces.append(img)
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions

def load_ferplus():
    train_path = os.path.join(ferplus_path, 'train')
    valid_path = os.path.join(ferplus_path, 'valid')
    test_path = os.path.join(ferplus_path, 'test')


def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x /255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

#**Model**

In [0]:
def vgg(input_shape, num_class):

    model = Sequential()

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(GlobalMaxPooling2D())
    model.add(Dense(num_classes, activation='softmax'))

model = vgg((64,64,1), 7)
model.summary()

In [0]:
def bkvgg12(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# **Training**

In [0]:
batch_size = 128
num_epochs = 10000
width, height = 64, 64
input_shape = (width, height, 1)
num_class = 7
patience = 50
base_path = 'models/'
model_name = 'bkvgg12'
model = bkvgg12(input_shape, num_class)

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience + 10)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.2, patience=int(patience/5), verbose=1)
model_path = base_path + model_name + '.{epoch:03d}-{val_acc:2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_path, 'val_acc', verbose=1, save_best_only=True)
call_backs = [model_checkpoint, csv_logger, early_stop, reduce_lr]
#call_backs = [model_checkpoint, csv_logger]

faces, emotions = load_fer2013((width, height))
faces = preprocess_input(faces)

num_sample, num_class = emotions.shape
x_train, x_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.2, shuffle=True)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, shuffle=True)

history = model.fit(x_train, y_train, validation_data=(x_val, y_val), 
              batch_size=batch_size,
              epochs=num_epochs,
              verbose=1,
              shuffle=True,
              callbacks=call_backs
              )


# **Draw chart**

In [0]:
history = pd.read_csv('models/_emotion_training.log')

# Plot training & validation accuracy values
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

#**Testing**

In [0]:
model_path='/content/models/bkvgg12.059-0.663695.hdf5'
model = load_model(model_path, compile=True)
score = model.evaluate(x_test, y_test)
print("test_loss:", round(score[0],4), '-', 'test_acc', round(score[1],4))

#**Report**

In [0]:
y_pred = model.predict(x_test, batch_size=32)
Y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(y_test, axis=1)
confu = confusion_matrix(Y_test, Y_pred)
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
fig, ax = plot_confusion_matrix(conf_mat=confu,
                                figsize=(8,8),
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True,
                                class_names=labels,
                                )
plt.show()
print(classification_report(Y_test, Y_pred, target_names=labels))